## 라이브러리 임포트

In [2]:
import re

## .txt 파일 추출

In [3]:
# 추출 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트\R078r3e\R078r3e_annex3_appendix1_unprocessed.txt'

# 텍스트 데이터 추출
with open(data_path, 'rb') as source:
    lines = source.readlines()
    lines = [element.decode('utf-8') for element in lines]
    
# 개행 문자 변환
pattern_doubleSpace = re.compile(r'\r\n')
lines = [pattern_doubleSpace.sub('\n', line) for line in lines]

# 메타 데이터 추출
if lines[0].startswith("@"):
    line_metaInfo = lines[0]
    lines.pop(0)    

text_sum = ''.join(lines)    
print(text_sum)

1.1. General:  
(a) The test is to establish a PBC for the vehicle type when being braked 
on the test surfaces described in Annex 3, paragraphs 1. 1.1. and 1.1.2.  
(b) The test comprises a number of stops with varying brake control forces.  
Both wheels shall be braked simultaneously up to the point reached 
before wheel lock, in order to achieve the maximum vehicle 
deceleration rate on the given test s urface.  
(c) The maximum vehicle deceleration rate is the highest value recorded 
during all the test stops.  
(d) The Peak Braking Coefficient (PBC) is calculated from the test stop that 
generates the maximum vehicle deceleration rate, as follows:   
(e) The value of PBC shall be rounded to two decimal places.  
1.2. Vehicle condition:  
(a) The test is applicable to all vehicle categories.   
(b) The anti -lock system shall be either disconnected or inoperative (ABS 
function disabled), between 40 km/h and 20 km/h.  
(c) Lightly loaded.  
(d) Engine disconnected.  
1.3. Test cond

## 필요 함수 정의

In [6]:
def lineParser(line, break_pattern, sel_group, opt_strip=False):
    breaks = re.search(break_pattern, line)
    
    parsed_result = []   
    for idx in sel_group:
        if opt_strip:
            parsed_result.append(breaks.group(idx).strip())
        else:
            parsed_result.append(breaks.group(idx))
            
    return(parsed_result)

In [7]:
def addressParser(address):
    # Define the regex pattern
    pattern = r'\s*(\d+)(?:\.(\d+))?(?:\.(\d+))?(?:\.(\d+))?(?:\.(\d+))?.*'
    
    # Match the pattern to the sentence
    match = re.match(pattern, address)
    
    if match:
        # Extract the matched groups
        chapter = match.group(1)
        article = match.group(2)
        paragraph = match.group(3)
        sub_paragraph = match.group(4)
        sub_sub_paragraph = match.group(5)
        
        # Create the result dictionary
        result = {
            'Chapter': chapter,
            'Article': article,
            'Paragraph': paragraph,
            'Sub-paragraph': sub_paragraph,
            'Sub-sub-paragraph': sub_sub_paragraph,
        }
        
        # Filter out None values
        result = {k: int(v) for k, v in result.items() if v is not None}
        
        return result
    else:
        return None


## 주석 테깅

In [11]:
# 주석 제거 (미사용)
 ## 주석 시작 행 확인
pattern_startAnnotation = r'^\s\d{1,}\s'
idxl_startAnnotation = [index for index, line in enumerate(lines) if re.match(pattern_startAnnotation, line)] 
 
pattern_endPage = r'^\[END OF PAGE\]'
idxl_endPage = [index for index, line in enumerate(lines) if re.match(pattern_endPage, line)]

pattern_onlyEnter = r'^\s*\r?\n$'

idxl_endAnnotation = idxl_endPage
idxl_endAnnotation.sort()

 ## 주석 종료 행 확인 및 범위 특정
idxl_remove = []
for idx_start in idxl_startAnnotation:
    idx_cursor = idx_start
    
    while(idx_cursor not in idxl_endAnnotation):
        idxl_remove.append(idx_cursor)
        idx_cursor = idx_cursor + 1
        
 ## 주석 제거
lines_processed = lines.copy()
for idx_remove in sorted(idxl_remove, reverse=True):
    lines_processed.pop(idx_remove)

In [8]:
# 주석 테깅

lines_processed = lines.copy()

pattern_startAnnotation = r'^\s?(\d{1,}\s.*)'
pattern_onlyEnter = r'^\s*\r?\n$'
pattern_endPage = r'^\[END OF PAGE\]'
idxl_endPage = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]

idx_startPage = 0
for idx_endPage in idxl_endPage:
   
   idxL_onlyEnter_inPage = []
   for idx_cusor in range(idx_endPage,idx_startPage-1,-1):
      if re.match(pattern_onlyEnter, lines_processed[idx_cusor]): idxL_onlyEnter_inPage.append(idx_cusor)
      
   if idxL_onlyEnter_inPage:
      
      if len(idxL_onlyEnter_inPage) > 2: idxL_onlyEnter_inPage = idxL_onlyEnter_inPage[0:2]
      
      for idx_cusor in range(min(idxL_onlyEnter_inPage),idx_endPage):
         if re.match(pattern_startAnnotation, lines_processed[idx_cusor]): lines_processed[idx_cusor] = '*' + lineParser(lines_processed[idx_cusor], pattern_startAnnotation, [1])[0]
         
   idx_startPage = idx_endPage
   
   

In [21]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

1. General  
1.1. Test surfaces  
1.1.1.  High friction surface:  
(a) Applicable to all dynamic brake tests excluding the ABS tests where a 
low-friction surface is specified;  
(b) The test area is a clean and level surface, with a gradient ≤   1 per cent;  
(c) The surface has a nominal peak braking coefficient (PBC) of 0.9, unless 
otherwise specified.  
1.1.2.  Low friction surface:  
(a) Applicable to all dynamic brake tests where a low -friction surface is 
specified;  
(b) The test area is a clean and  level surface, with a gradient  ≤ 1  per cent;  
(c) The surface has a PBC of ≤  0.45.  
1.1.3.  Measurement of PBC:  
The PBC is measured as deter mined by the approval Type Approval A uthority 
using either:  
(a) An ASTM International (ASTM) E1136 -93 (Re -approved 2003) 
standard reference test tyre, in accordance with ASTM 
Method  E1337 -90 (Re -approved  2008 ), at a speed of 40 mph; or  
(b) The method speci fied in the Appendix 1 to this annex:  
1.1.4.  Parking brake sy

## 특수 데이터 제거 (이미지, 테이블, 수식) (현제 불안정, 미사용)

In [16]:
# 특수 데이터 제거
 ## 특수 데이터 시작 시점 확인
pattern_endWithColon = r".*\:\s{1,}\n$"
idxL_endWithColon = [index for index, line in enumerate(lines_processed) if re.match(pattern_endWithColon, line)]

pattern_endPage = r'^\[END OF PAGE\]'
idxl_endPage = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]

pattern_numnering = r'^\s*(\d{1,}\.){1,}'
idxl_numnering = [index for index, line in enumerate(lines_processed) if re.match(pattern_numnering, line)]


pattern_item = r"^\(.*\)"
idxl_item = [index for index, line in enumerate(lines_processed) if re.match(pattern_item, line)]

idxL_cursorStart = [idx_endWithColon + 1 for idx_endWithColon in idxL_endWithColon]

idxL_cursorEnd = idxl_numnering + idxl_item + idxl_endPage
idxL_cursorEnd.sort()
 
 ## 특수 데이터 종료 시점 확인 및 데이터 범위 특정
idxL_remove = []
for idx_cursorStart in idxL_cursorStart:
    
     ### 시작 지점부터 한 줄씩 종료 조건을 확인하며 데이터 범위 탐색
    idx_cursor = idx_cursorStart
    #print(f"Start with the line IDX:{idx_moment}")
    
    while idx_cursor not in idxL_cursorEnd:
        idxL_remove.append(idx_cursor)
        #print(f" Mark the line IDX to be removed:{idx_moment}")
        
        idx_cursor = idx_cursor + 1
        
     ### 포함 여부 확인 후 작업 스케줄 수정 (특수 데이터의 범위로 특정된 부분 ~ 기존에 확인한 탐색 시작 지점)
    idxL_redundant = [ idxL_cursorStart.index(elm_redundant) for elm_redundant in list(filter(lambda x : (idx_cursorStart < x <= idx_cursor), idxL_cursorStart)) ]
    for idx_redundant in sorted(idxL_redundant, reverse=True):
        idxL_cursorStart.pop(idx_redundant)
         #print(f"Pop out the start IDX of the redundant: {idxL_start.pop(idx_redundant)}")

 ## 특정한 범위를 바탕으로 특수 데이터 제거        
for idx_remove in sorted(idxL_remove, reverse=True):
    lines_processed.pop(idx_remove)

In [17]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

1. General  
1.1. Test surfaces  
1.1.1.  High friction surface:  
(a) Applicable to all dynamic brake tests excluding the ABS tests where a 
low-friction surface is specified;  
(b) The test area is a clean and level surface, with a gradient ≤   1 per cent;  
(c) The surface has a nominal peak braking coefficient (PBC) of 0.9, unless 
otherwise specified.  
1.1.2.  Low friction surface:  
(a) Applicable to all dynamic brake tests where a low -friction surface is 
specified;  
(b) The test area is a clean and  level surface, with a gradient  ≤ 1  per cent;  
(c) The surface has a PBC of ≤  0.45.  
1.1.3.  Measurement of PBC:  
(a) An ASTM International (ASTM) E1136 -93 (Re -approved 2003) 
standard reference test tyre, in accordance with ASTM 
Method  E1337 -90 (Re -approved  2008 ), at a speed of 40 mph; or  
(b) The method speci fied in the Appendix 1 to this annex:  
1.1.4.  Parking brake system tests:  
1.1.5.  Test lane width:  
2.5 m. 
 For three -wheeled and four -wheeled vehicl

## 띄어쓰기 제거

In [9]:
# 띄어쓰기 제거: 페이지 전환 부분

 ## 페턴 
pattern_withSpace = r'^\s+'
pattern_endPage = r'^\[END OF PAGE\]'

 ## 페이지 종료 시점 확인
idxL_pageEnd = [index for index, line in enumerate(lines_processed) if re.match(pattern_endPage, line)]
idxL_pageEnd.pop()

 ## 페이지 종료 후 띄어쓰기 삽입 여부 확인 및 수정
for idx_pageEnd in idxL_pageEnd:
    if re.match(pattern_withSpace, lines_processed[idx_pageEnd+1]):
        lines_processed[idx_pageEnd+1] = lines_processed[idx_pageEnd+1][1:]

# 띄어쓰기 제거: 항목 번호 시작 전 띄어쓰기
 
pattern_numbering_withSpcae = r'^\s+((?:\d{1,}\.){1,}.*)'
idxl_numbering_withSpcae = [index for index, line in enumerate(lines_processed) if re.match(pattern_numbering_withSpcae, line)]

for idx_withSpace_exceptions in idxl_numbering_withSpcae:
    lines_processed[idx_withSpace_exceptions] = lineParser(lines_processed[idx_withSpace_exceptions], pattern_numbering_withSpcae, [1])[0] + '\n'

## 단독 개행('\r\n') 및 페이지 전환('[END OF PAGE]') 제거

In [10]:
# 단독 개행 및 페이지 전환 제거

## 패턴
pattern_onlyEnter = r'^\s*\r?\n$'
pattern_endPage = r'^\[END OF PAGE\]'

## 패턴과 일치하는 행 확인
idxL_onlyEnter_endPage = [index for index, line in enumerate(lines_processed) if (re.match(pattern_onlyEnter, line) or re.match(pattern_endPage, line))]

## 패턴과 일치하는 행 제거
for idx_remove in sorted(idxL_onlyEnter_endPage, reverse=True):
    lines_processed.pop(idx_remove)

In [11]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_processed)
print(text_sum)

1.1. General:  
(a) The test is to establish a PBC for the vehicle type when being braked 
on the test surfaces described in Annex 3, paragraphs 1. 1.1. and 1.1.2.  
(b) The test comprises a number of stops with varying brake control forces.  
Both wheels shall be braked simultaneously up to the point reached 
before wheel lock, in order to achieve the maximum vehicle 
deceleration rate on the given test s urface.  
(c) The maximum vehicle deceleration rate is the highest value recorded 
during all the test stops.  
(d) The Peak Braking Coefficient (PBC) is calculated from the test stop that 
generates the maximum vehicle deceleration rate, as follows:   
(e) The value of PBC shall be rounded to two decimal places.  
1.2. Vehicle condition:  
(a) The test is applicable to all vehicle categories.   
(b) The anti -lock system shall be either disconnected or inoperative (ABS 
function disabled), between 40 km/h and 20 km/h.  
(c) Lightly loaded.  
(d) Engine disconnected.  
1.3. Test cond

## 행 병합

In [12]:
pattern_numnering = r'^\s*(\d{1,}\.){1,}'
pattern_items = r"^\(.{1,3}\)"
pattern_text_withIndent = r"\s+"
pattern_startAnnotation = r'^\*\d{1,}\s.*'

idxL_normalText = [index for index, line in enumerate(lines_processed) if not (re.match(pattern_items, line)) and not (re.match(pattern_numnering, line))]
idxL_normalText = [index for index in idxL_normalText if not (re.match(pattern_text_withIndent, lines_processed[index]))]
idxL_normalText = [index for index in idxL_normalText if not (re.match(pattern_startAnnotation, lines_processed[index]))]

for idx_normalText in sorted(idxL_normalText, reverse=True):
    lines_processed[idx_normalText-1] = lines_processed[idx_normalText-1][:-2]
    
text_sum = ''.join(lines_processed)
lines_merged = text_sum.split('\n')
lines_merged = [line + '\n' for line in lines_merged]

### 예외처리: 챕터 제목 병합

In [13]:
# 챕터 제목 행 병합
pattern_chapterTitle = r'^\s{2}[a-zA-Z]'
idxL_chapterTitle = [index for index, line in enumerate(lines_merged) if (re.match(pattern_chapterTitle, line))]

for idx_chapterTitle in sorted(idxL_chapterTitle, reverse=True):
    lines_merged[idx_chapterTitle-1] = lines_merged[idx_chapterTitle-1][:-4]

## 이중 띄어쓰기 제거

In [14]:
pattern_doubleSpace = re.compile(r' {2,}')
lines_merged = [pattern_doubleSpace.sub(' ', line) for line in lines_merged]

In [15]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_merged)
print(text_sum)

1.1. General: 
(a) The test is to establish a PBC for the vehicle type when being brakedon the test surfaces described in Annex 3, paragraphs 1. 1.1. and 1.1.2. 
(b) The test comprises a number of stops with varying brake control forces. Both wheels shall be braked simultaneously up to the point reachedbefore wheel lock, in order to achieve the maximum vehicledeceleration rate on the given test s urface. 
(c) The maximum vehicle deceleration rate is the highest value recordedduring all the test stops. 
(d) The Peak Braking Coefficient (PBC) is calculated from the test stop thatgenerates the maximum vehicle deceleration rate, as follows: 
(e) The value of PBC shall be rounded to two decimal places. 
1.2. Vehicle condition: 
(a) The test is applicable to all vehicle categories. 
(b) The anti -lock system shall be either disconnected or inoperative (ABSfunction disabled), between 40 km/h and 20 km/h. 
(c) Lightly loaded. 
(d) Engine disconnected. 
1.3. Test conditions and procedure: 
(a) 

## 행 번호 점검 (예외 처리)

In [30]:
pattern_items = r"^\(.*\)"
pattern_startAnnotation = r'^\*\d{1,}\s.*'
pattern_text_withIndent = r"\s+"
address_pre = list(addressParser(lines_merged[0]).values())

idxl_exceptions = [] 

for idx, line in enumerate(lines_merged[1:8]):
    if re.match(pattern_items, line): continue
    if re.match(pattern_startAnnotation, line): continue
    if re.match(pattern_text_withIndent, line): continue
    
    flag_exception = 0
    address_inCursor = list(addressParser(line).values())
    
    if len(address_inCursor) > len(address_pre):  
       diffs = [inCursor - pre for pre, inCursor in zip(address_pre, address_inCursor[:len(address_pre)])]
       for diff in diffs:
           if diff != 0: flag_exception = 1
           
    
    elif len(address_inCursor) <= len(address_pre):
        diffs = [inCursor - pre for pre, inCursor in zip(address_pre[:len(address_inCursor)], address_inCursor)]
        for diff in diffs[:-1]:
           if diff != 0: flag_exception = 1
        if diffs[-1] != 1 : flag_exception = 1
        
    if flag_exception:
        idxl_exceptions.append((idx, line))
    else:
        address_pre = address_inCursor
    
    flag_exception = 0

In [31]:
# 테스트 - 가공 텍스트 출력
text_sum = ''.join(lines_merged)
print(text_sum)

1. General 
1.1. Test surfaces 
1.1.1. High friction surface: 
(a) Applicable to all dynamic brake tests excluding the ABS tests where alow-friction surface is specified; 
(b) The test area is a clean and level surface, with a gradient ≤ 1 per cent; 
(c) The surface has a nominal peak braking coefficient (PBC) of 0.9, unlessotherwise specified. 
1.1.2. Low friction surface: 
(a) Applicable to all dynamic brake tests where a low -friction surface isspecified; 
(b) The test area is a clean and level surface, with a gradient ≤ 1 per cent; 
(c) The surface has a PBC of ≤ 0.45. 
1.1.3. Measurement of PBC: The PBC is measured as deter mined by the approval Type Approval A uthorityusing either: 
(a) An ASTM International (ASTM) E1136 -93 (Re -approved 2003)standard reference test tyre, in accordance with ASTMMethod E1337 -90 (Re -approved 2008 ), at a speed of 40 mph; or 
(b) The method speci fied in the Appendix 1 to this annex: 
1.1.4. Parking brake system tests: The specified test slope s 

## .txt 파일 저장

In [16]:
# 결과 저장 경로
data_path=r'G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트\R078r3e\R078r3e_annex3_appendix1_processed.txt'
# txt 파일 저장
with open(data_path, 'w', encoding='utf-8') as result:
    text_sum = ''.join(lines_merged)    
    result.write(text_sum)